[View in Colaboratory](https://colab.research.google.com/github/thayumaanavan/Deep-Reinforcement-Learning/blob/master/Deep_Q_Learning_Keras.ipynb)

In [1]:
!pip install gym

  Running setup.py bdist_wheel for gym: started
  Running setup.py bdist_wheel for gym: finished with status 'done'
  Stored in directory: C:\Users\chitrara\AppData\Local\pip\Cache\wheels\cb\14\71\f4ab006b1e6ff75c2b54985c2f98d0644fffe9c1dddc670925
  Running setup.py bdist_wheel for future: started
  Running setup.py bdist_wheel for future: finished with status 'done'
  Stored in directory: C:\Users\chitrara\AppData\Local\pip\Cache\wheels\bf\c9\a3\c538d90ef17cf7823fa51fc701a7a7a910a80f6a405bf15b1a
Successfully built gym future
  Found existing installation: certifi 2016.2.28
    Uninstalling certifi-2016.2.28:
      Successfully uninstalled certifi-2016.2.28


  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [4]:
!pip install keras
from keras.models import Sequential      # One layer after the other
from keras.layers import Dense, Flatten  # Dense layers are fully connected layers, Flatten layers flatten out multidimensional inputs
from collections import deque            # For storing moves 

import numpy as np
import gym                                # To train our network
env = gym.make('MountainCar-v0')          # Choose game (any in the gym should work)

import random     # For sampling batches from the observations

  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.
Using TensorFlow backend.


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [27]:
model = Sequential()
model.add(Dense(20, input_shape=(2,) + env.observation_space.shape, kernel_initializer='uniform', activation='relu'))
model.add(Flatten())
model.add(Dense(18, kernel_initializer='uniform', activation = 'relu'))
model.add(Dense(10, kernel_initializer='uniform', activation = 'relu'))
model.add(Dense(env.action_space.n, kernel_initializer='uniform', activation='linear'))

model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

D = deque()

observetime = 10000
epsilon = 0.2
gamma = 0.9
mb_size = 1000


In [28]:
observation = env.reset()
obs = np.expand_dims(observation, axis=0)
state = np.stack((obs,obs), axis=1)
done = False

for t in range(observetime):
  env.render()
  if np.random.rand() <= epsilon:
    action = np.random.randint(0, env.action_space.n, size=1)[0]
  else:
    Q = model.predict(state)
    action = np.argmax(Q)
  
  observation_new, reward, done, info = env.step(action)
  obs_new = np.expand_dims(observation_new, axis=0)
  state_new = np.append(np.expand_dims(obs_new, axis=0), state[:,:1,:], axis=1)
  D.append((state, action, reward, state_new, done))
  
  state = state_new
  if done:
    env.reset()
    obs = np.expand_dims(observation,axis=0)
    state = np.stack((obs,obs), axis=1)
 
print("Observing Finished") 

Observing Finished


In [29]:
minibatch = random.sample(D, mb_size)

input_shape = (mb_size, ) + state.shape[1:]
inputs = np.zeros(input_shape)
targets = np.zeros((mb_size, env.action_space.n))

for i in range(0,mb_size):
  state = minibatch[i][0]
  action = minibatch[i][1]
  reward = minibatch[i][2]
  state_new = minibatch[i][3]
  done = minibatch[i][4]
  
  inputs[i:i+1] = np.expand_dims(state, axis=0)
  targets[i] = model.predict(state)
  Q_sa = model.predict(state_new)
  
  if done:
    targets[i, action] = reward
  else:
    targets[i, action] = reward + gamma * np.max(Q_sa)
  
  
  model.train_on_batch(inputs, targets)

print("Learning Finished")

Learning Finished


In [31]:
observation = env.reset()
obs = np.expand_dims(observation, axis=0)
state = np.stack((obs,obs), axis =1)
done = False
total_rewards = 0.0

while not done:
  env.render()
  Q = model.predict(state)
  action = np.argmax(Q)
  observation, reward, done, info = env.step(action)
  obs = np.expand_dims(observation, axis=0)
  state = np.append(np.expand_dims(obs, axis=0), state[:,:1,:], axis=1)
  total_rewards += reward
print('Game Ended... Rewards:'+ str(retotal_rewardsward))  

Game Ended... Rewards:-200.0
